## The SEI3R Model

Another extension of SIR, designed during the paper we read for this work, https://arxiv.org/pdf/2003.13221.pdf, Planning as Inference in Epedimiological Control Models. 

### Model Dynamics


A member of the susceptible population $S$ moves to the exposed $E$ after being exposed to an infections person, where "exposure" is defined as the previously susceptible person contracting the illness. 

After an incubation period, a random duration parameterized by $\alpha$, the individual develops a mild infection $I_1$, from here, the indivdual either moves to recover $R$, or progress to a severe infection $I_2$. At $I_2$ they again either recover, or progress to critical infection $I_3$. From $I_3$, the critically ill individual will either die or recover. 

The summed population of all compartments at time $t$ is $N_t$. For the purposes of are work, we keep this value fixed. 

The dynamics of this system are defined by the following equations:


$$\frac{d}{dt}S_t = - (1 - u) \frac{1}{N_t} \sum_{i=1}^3 \beta_i I_{i,t}S_t$$

$$\frac{d}{dt}E = (1 - u) \frac{1}{N_t} \sum_{i=1}^3 \beta_i I_{i,t}S_t - \alpha E_t $$

$$\frac{d}{dt} I_{1,t} = \alpha E_t -p_1 I_{1,t} - \gamma_1 I_{1,t}$$


$$\frac{d}{dt} I_{2,t} =  p_1 I_{1,t} -p_2 I_{2,t} - \gamma_2 I_{2,t}$$


$$\frac{d}{dt} I_{3,t} =  p_2 I_{2,t} -p_3 I_{3,t} - \gamma_3 I_{3,t}$$


$$\frac{d}{dt} R_{t} = \gamma_1 I_{1,t} + \gamma_2 I_{2,t} +  \gamma_3 I_{3,t}$$





For simulations with this model, we initialize with $0.01 \%$ of the population as exposed, and the remaining $99.99 \%$ as susceptible. 

In [3]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.epidemiology import CompartmentalModel, binomial_dist, infection_dist

%matplotlib inline
#assert pyro.__version__.startswith('1.5.0') # I have 1.5.1, hopefully not a problem to comment this out. 
torch.set_default_dtype(torch.double)  # Required for MCMC inference.
pyro.enable_validation(True)  # Always a good idea.
smoke_test = ('CI' in os.environ)

In [ ]:

# check https://github.com/SwapneelM/covid/blob/master/SEIR/seir.py for help. 
class SimpleSEI3RModel(CompartmentalModel):
    """
    Susceptible-Exposed-Infected1-Infected2-Infected3-Recovered-(Dead) model.

    This is a stochastic discrete-time discrete-state model with seven
    compartments: "S" for susceptible, "E" for exposed, "I" for infected (with 3 degrees of illness), "D"
    for deceased individuals, and "R" for recovered individuals (the recovered
    individuals are implicit: ``R = population - S - E - I1 - I2 - I3 - D``) with
    transitions ``S -> E -> I1 -> R``,  ``S -> E -> I1 -> I2 -> R``,  ``S -> E -> I1 -> -> I2 -> I3 -> R``
    and ``I3 -> D``.

    Because the transitions are not simple linear succession, this model
    implements a custom :meth:`compute_flows()` method.

    :param int population: Total ``population = S + E + I3 + R + D``.
    :param float incubation_time: Mean incubation time (duration in state
        ``E``). Must be greater than 1.
    :param float recovery_time: Mean recovery time (duration in state
        ``I``). Must be greater than 1.
    :param float mortality_rate: Portion of infections resulting in death.
        Must be in the open interval ``(0, 1)``.
    :param iterable data: Time series of new observed infections. Each time
        step is Binomial distributed between 0 and the number of ``S -> E``
        transitions. This allows false negative but no false positives.
    """

    def __init__(self, population, incubation_time, recovery_time,
                 mortality_rate, data):
        compartments = ("S", "E", "I", "I2", "I3", "R")  # D is implicit.
        duration = len(data)
        super().__init__(compartments, duration, population)

        assert isinstance(incubation_time, float)
        assert incubation_time > 1
        self.incubation_time = incubation_time

        assert isinstance(recovery_time, float)
        assert recovery_time > 1
        self.recovery_time = recovery_time

        assert isinstance(mortality_rate, float)
        assert 0 < mortality_rate < 1
        self.mortality_rate = mortality_rate

        self.data = data

    def global_model(self):
        tau_e = self.incubation_time
        tau_i = self.recovery_time
        mu = self.mortality_rate
        R0 = pyro.sample("R0", dist.LogNormal(0., 1.))
        rho = pyro.sample("rho", dist.Beta(10, 10))
        return R0, tau_e, tau_i, mu, rho

    def initialize(self, params):
        # Start with a single infection ( at stage 1) .
        # pretty ambiguous. Check with paper see if they do something different. 
        return {"S": self.population - 1, "E": 0, "I": 1, "I2":0, "I3":0, "R": 0}

    def transition(self, params, state, t):
        """
        How we go from one compartment to another. After definining changes we update compartments. 
        
        """
        R0, tau_e, tau_i, mu, rho = params # add the other ones from paper
        p1 = .5
        p2 = .5
        gamma1 = .2
        gamma2 = .3
        gamma3 = .5
        kappa = 0.005

        # Sample flows between compartments.
        S2E = pyro.sample("S2E_{}".format(t),
                          infection_dist(individual_rate=R0 / tau_i,
                                         num_susceptible=state["S"],
                                         num_infectious=state["I"],
                                         population=self.population))
        
        E2I = pyro.sample("E2I_{}".format(t),
                          binomial_dist(state["E"], 1 / tau_e))
        
        I2I2 = pyro.sample("I2I2_{}".format(t),
                          binomial_dist(state["I"], p1))
        
        I2R = pyro.sample("I2R_{}".format(t),
                          binomial_dist(state["I"], gamma1))
       
        I22I3 = pyro.sample("I22I3_{}".format(t),
                          binomial_dist(state["I2"], p2))
        
        I22R = pyro.sample("I22R_{}".format(t),
                          binomial_dist(state["I2"], gamma2))
        
        
        I32R = pyro.sample("I32R_{}".format(t),
                          binomial_dist(state["I3"], gamma3))
        
        I32D = pyro.sample("I32D_{}".format(t),
                          binomial_dist(state["I3"], kappa))
        
        
        # Of the 1/tau_i expected recoveries-or-deaths, a portion mu die and
        # the remaining recover. Alternatively we could model this with a
        # Multinomial distribution I2_ and extract the two components I2D and
        # I2R, however the Multinomial distribution does not currently
        # implement overdispersion or moment matching.
        

        # update compartments. 
        state["S"] = state["S"] - S2E
        state["E"] = state["E"] + S2E - E2I
        state["I"] = state["I"] + E2I - I2R - I2I2
        state["I2"] = state["I2"] + I2I2 - I22R - I22I3
        state["I3"] = state["I3"] + I22I3 - I32R - I32D
        state["R"] = state["R"] + I32R + I2R + I22R
        
        
        
        

        # Condition on observations. 
        t_is_observed = isinstance(t, slice) or t < self.duration
        pyro.sample("obs_{}".format(t),
                    binomial_dist(S2E, rho),
                    obs=self.data[t] if t_is_observed else None)

    def compute_flows(self, prev, curr, t):
        """
        Having a tough time making these line up. Need to choose one to define first? 
        
        Should a new variable be implicit? 
        """
        
        
        # 
        
        # First part, flow through first parts
        S2E = prev["S"] - curr["S"]  # S can only go to E.
        E2I = prev["E"] - curr["E"] + S2E
        
        
        # Secondly, flow through I leaving to go to I2 or R
        
        I2 = prev["I"] - curr["I"] + E2I 
        
        
        
        # Next is I2I2, I22R
        
        # Here is where I get stuck. I cannot define either of these without specifying one first. 
        I2R = prev["I"] - curr["I"] + E2I - I2I2

        I2I2 = prev["I"] - curr["I"] + E2I - I2R

        

        I32D = curr["D"] - prev["D"]  # D can only have come from I3. 
        II32R = prev["I3"] - curr["I3"] + I22I3 - I32D

        #again, stuck I22I3 is also defined weirdly
        
        
        # flow is change in I2 plus incoming minus recovered subset. 
        I22I3 = prev["I2"] - curr["I2"] + I2I2 - I22R 
        
        I22R = prev["I2"] - curr["I2"] + I2I2 - I22I3
        
        

        
        
        return {
            "S2E_{}7".format(t): S2E,
            "E2I_{}".format(t): E2I,
            "I2I2_{}".format(t): I2I2,
            "I22I3_{}".format(t): I22I3,

            "I32D_{}".format(t): I32D,
            "I2R_{}".format(t): I2R,
            "I22R_{}".format(t): I22R,
            "I32R_{}".format(t): I32R,

        }

